<a href="https://colab.research.google.com/github/zskylarli/transcrobialGAN/blob/main/transcrobial_pix2pix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transcrobial GAN

*PyTorch implementation*

Last updated: 4/14/2021

Author: Skylar Li 

Used datasets:

- https://www.dropbox.com/sh/g8w03sfarsg979b/AAB2cVzUdabKH-NdAT1YBevIa?dl=0



# Install

In [42]:
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

Cloning into 'pytorch-CycleGAN-and-pix2pix'...
remote: Enumerating objects: 2447, done.
remote: Total 2447 (delta 0), reused 0 (delta 0), pack-reused 2447
Receiving objects: 100% (2447/2447), 8.18 MiB | 13.75 MiB/s, done.
Resolving deltas: 100% (1535/1535), done.


In [43]:
import os
os.chdir('pytorch-CycleGAN-and-pix2pix/')

In [44]:
!pip install -r requirements.txt

     |████████████████████████████████| 676 kB 6.3 MB/s 
     |████████████████████████████████| 1.7 MB 46.5 MB/s 
     |████████████████████████████████| 53 kB 1.9 MB/s 
     |████████████████████████████████| 144 kB 47.0 MB/s 
     |████████████████████████████████| 180 kB 50.6 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
  Created wheel for visdom: filename=visdom-0.1.8.9-py3-none-any.whl size=655250 sha256=9c45632f74e661d420cd9b2e7dbcb0d72fc69e9cac86c7f253f7508a0abec3c0
  Stored in directory: /root/.cache/pip/wheels/2d/d1/9b/cde923274eac9cbb6ff0d8c7c72fe30a3da9095a38fd50bbf1
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=4244670c6901efc66fb910e697fdf733d957f074427bf72207b14e072a1dc96b
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
  Created wheel for torchfile: filename=torchfile-0.1.0-py3-none-any.whl size=5709 sha256=180337e01b7728f787f118ad9ebd99015baf4b3d43674

# Dataset

Upload dataset to Google Drive folder. 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
PATH_OF_DATA= '/content/drive/MyDrive/transcrobialGAN'
!ls {PATH_OF_DATA}

 021622_test   orginal_images  'Transcrobial Images Dataset Sorting.gsheet'


In [37]:
import os

old_path = '/content/drive/MyDrive/transcrobialGAN/02162022_old'
new_path = '/content/drive/MyDrive/transcrobialGAN/02162022_new'

list_of_files = []
with os.scandir(old_path) as it:
  for entry in it:
    if entry.is_file():
      list_of_files.append(entry.name)

print(len(list_of_files))

162


In [38]:
from PIL import Image

counter = 1;
for file in list_of_files:
  usage = '';

  #open file
  f = Image.open(os.path.join(old_path,file))
  file = file.split('.jpg')
  file = file[0] + '.png'

  if counter % 2 == 0:
    usage = 'A';
  else:
    usage = 'B';
  
  #save file
  folder_path = os.path.join(new_path, usage)
  if not os.path.isdir(folder_path):
    os.mkdir(folder_path)
  f = f.save(os.path.join(folder_path,file))

  counter += 1;

In [39]:
import numpy as np
import shutil

def split_train_test(path_name, split_ratio):
  rootdir = new_path + '/' + path_name;

  allFileNames = os.listdir(rootdir)

  np.random.shuffle(allFileNames)

  test_ratio = split_ratio

  train_FileNames, test_FileNames = np.split(np.array(allFileNames),
                                                        [int(len(allFileNames)* (1 - test_ratio))])

  train_FileNames = [rootdir+'/'+ name for name in train_FileNames.tolist()]
  test_FileNames = [rootdir+'/' + name for name in test_FileNames.tolist()]

  print('Total images: ', len(allFileNames))
  print('Training: ', len(train_FileNames))
  print('Testing: ', len(test_FileNames))

  i = 1;
  j = len(train_FileNames)+1;

  os.makedirs(rootdir +'/train/')
  os.makedirs(rootdir +'/test/')

  for name in train_FileNames:
    new_name = rootdir + '/train/' + str(i) + '.png';
    os.rename(name, new_name)
    i += 1;

  for name in test_FileNames:
    new_name = rootdir + '/test/' + str(j) + '.png';
    os.rename(name, new_name)
    j += 1;

In [40]:
split_train_test('A',0.3)

Total images:  81
Training:  56
Testing:  25


In [41]:
split_train_test('B',0.3)

Total images:  81
Training:  56
Testing:  25


In [66]:
import cv2

def resize_img(img):
  width = 790
  height = 900
  dim = (width, height)
  resized = cv2.resize(img, dim)
  return resized

def image_write(path_A, path_B, path_AB):
    im_A = cv2.imread(path_A, 1) 
    im_A = resize_img(im_A)
    im_B = cv2.imread(path_B, 1) 
    im_B = resize_img(im_B)
    im_AB = np.concatenate([im_A, im_B], 1)

    cv2.imwrite(path_AB, im_AB)

In [81]:
def comb_images(path):
  path_A_train = path + '/A/train/'
  path_B_train = path + '/B/train/'
  path_A_test = path + '/A/test/'
  path_B_test = path + '/B/test/'
  path_AB_train = path + '/AB/train'
  path_AB_test = path + '/AB/test'
  trainFiles = os.listdir(path_A_train)
  testFiles = os.listdir(path_A_test)
  train_num = len(trainFiles)
  test_num = len(testFiles)

  for i in range(1, train_num+1):
    img_path_A_train = path_A_train + '/' + str(i) + '.png'
    img_path_B_train = path_B_train + '/' + str(i) + '.png'
    img_path_AB_train = path_AB_train + '/' + str(i) + '_comb.png'
    image_write(img_path_A_train, img_path_B_train, img_path_AB_train)
  
  for j in range(train_num+1, train_num+test_num+1):
    img_path_A_test = path_A_test + '/' + str(j) + '.png'
    img_path_B_test = path_B_test + '/' + str(j) + '.png'
    img_path_AB_test = path_AB_test + '/' + str(j) + '_comb.png'
    image_write(img_path_A_test, img_path_B_test, img_path_AB_test)

In [82]:
comb_images(new_path)

# Training

-   `python train.py --dataroot /content/gdrive/"My Drive"/finalproject/AB --name dibas_pix2pix --model pix2pix --direction BtoA`

Change the `--dataroot` and `--name` to your own dataset's path and model's name. Use `--gpu_ids 0,1,..` to train on multiple GPUs and `--batch_size` to change the batch size. Add `--direction BtoA` if you want to train a model to transfrom from class B to A.

In [85]:
!python train.py --dataroot /content/drive/MyDrive/transcrobialGAN/02162022_new/AB --name transcrobial_pix2pix --model pix2pix --direction BtoA

----------------- Options ---------------
               batch_size: 1                             
                    beta1: 0.5                           
          checkpoints_dir: ./checkpoints                 
           continue_train: False                         
                crop_size: 256                           
                 dataroot: /content/drive/MyDrive/transcrobialGAN/02162022_new/AB	[default: None]
             dataset_mode: aligned                       
                direction: BtoA                          	[default: AtoB]
              display_env: main                          
             display_freq: 400                           
               display_id: 1                             
            display_ncols: 4                             
             display_port: 8097                          
           display_server: http://localhost              
          display_winsize: 256                           
                    epoch: lates

# Testing

-   `python test.py --dataroot ./datasets/facades --direction BtoA --model pix2pix --name facades_pix2pix`

Change the `--dataroot`, `--name`, and `--direction` to be consistent with your trained model's configuration and how you want to transform images.


In [86]:
!ls checkpoints/

transcrobial_pix2pix


In [87]:
!python test.py --dataroot /content/drive/MyDrive/transcrobialGAN/02162022_new/AB --direction BtoA --model pix2pix --name transcrobial_pix2pix

----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 256                           
                 dataroot: /content/drive/MyDrive/transcrobialGAN/02162022_new/AB	[default: None]
             dataset_mode: aligned                       
                direction: BtoA                          	[default: AtoB]
          display_winsize: 256                           
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
                  isTrain: False                         	[default: None]
                

# Save results to My Drive

In [88]:
!zip -r /content/drive/MyDrive/transcrobialGAN/02162022_new/results /content/pytorch-CycleGAN-and-pix2pix/results/transcrobial_pix2pix/test_latest/images

  adding: content/pytorch-CycleGAN-and-pix2pix/results/transcrobial_pix2pix/test_latest/images/ (stored 0%)
  adding: content/pytorch-CycleGAN-and-pix2pix/results/transcrobial_pix2pix/test_latest/images/79_comb_real_A.png (deflated 0%)
  adding: content/pytorch-CycleGAN-and-pix2pix/results/transcrobial_pix2pix/test_latest/images/61_comb_real_A.png (deflated 0%)
  adding: content/pytorch-CycleGAN-and-pix2pix/results/transcrobial_pix2pix/test_latest/images/72_comb_real_B.png (deflated 0%)
  adding: content/pytorch-CycleGAN-and-pix2pix/results/transcrobial_pix2pix/test_latest/images/66_comb_real_A.png (deflated 0%)
  adding: content/pytorch-CycleGAN-and-pix2pix/results/transcrobial_pix2pix/test_latest/images/80_comb_fake_B.png (deflated 0%)
  adding: content/pytorch-CycleGAN-and-pix2pix/results/transcrobial_pix2pix/test_latest/images/59_comb_real_B.png (deflated 0%)
  adding: content/pytorch-CycleGAN-and-pix2pix/results/transcrobial_pix2pix/test_latest/images/77_comb_fake_B.png (deflated 